In [72]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm, trange
from constants import end_date

In [73]:
df = pd.read_csv("data/price_data.csv")
smoke_test = False
if smoke_test:
    df = df[df["ticker"] == "AAPL"]


In [74]:
df_ts = df.pivot(index='date', columns='ticker', values='close')

In [75]:
stocks = df_ts.columns.tolist()

In [76]:
train, test = df_ts[df_ts.index <= end_date], df_ts[df_ts.index > end_date]


In [79]:
import warnings
from tqdm import tqdm

# Suppress statsmodels warnings about date index frequency
warnings.filterwarnings('ignore', category=FutureWarning, module='statsmodels')
warnings.filterwarnings('ignore', message='.*date index.*frequency.*', module='statsmodels')
warnings.filterwarnings('ignore', message='.*No supported index.*', module='statsmodels')

all_stock_predictions = {}
for s in tqdm(stocks, desc="Processing stocks", position=0):
    all_predictions = []
    current_data = train[s].copy()
    
    # Set frequency if it's a datetime index
    if hasattr(current_data.index, 'freq') and current_data.index.freq is None:
        current_data = current_data.asfreq('D')  # Adjust 'D' to your actual frequency
   
    for i in tqdm(range(len(test.index)), desc=f"Stock: {s}", position=1, leave=False):
        # Fit model on current available data
        model = ARIMA(current_data, order=(1, 0, 0)).fit()
        
        # Make one-step ahead prediction
        prediction = model.forecast(steps=1)
        all_predictions.append(prediction.values[0])
        
        # Update current_data with the ACTUAL observed value for next iteration
        if i < len(test.index) - 1:
            actual_value = test[s].iloc[i]
            new_data = pd.Series([actual_value], index=[test.index[i]])
            if hasattr(current_data.index, 'freq') and current_data.index.freq is not None:
                new_data = new_data.asfreq(current_data.index.freq)
            current_data = pd.concat([current_data, new_data])
   
    # Store predictions for this stock
    all_stock_predictions[s] = all_predictions

# Create DataFrame after the loop finishes
predictions_df = pd.DataFrame(data=all_stock_predictions, index=test.index)

# Re-enable warnings if needed
warnings.resetwarnings()

Processing stocks:   1%|          | 5/496 [01:39<2:58:19, 21.79s/it]c:\Users\feder\anaconda3\envs\llm\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\feder\anaconda3\envs\llm\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\feder\anaconda3\envs\llm\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\feder\anaconda3\envs\llm\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive param

In [80]:
predictions_df.to_csv("data/models/preds_arima.csv", index=False)